In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU name: Tesla T4


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.9 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from ultralytics import YOLO

def main():
  model = YOLO("yolov8n-seg.pt")

  model.train(
      data="/content/drive/MyDrive/please/data.yaml",
      epochs=300,
      imgsz=640,
      batch=8,
      freeze=10,
      patience=100,
      lr0=0.001,
      cos_lr=True,
      hsv_h=0.01,
      hsv_s=0.6,
      hsv_v=0.3,
      device=0,
  )

if __name__ == "__main__":
    main()

Ultralytics 8.4.16 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/please/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.01, hsv_s=0.6, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pe

In [13]:
import os

image_dir = "/content/drive/MyDrive/please/train/images"
label_dir = "/content/drive/MyDrive/please/train/labels"

for file in os.listdir(label_dir):
    path = os.path.join(label_dir, file)
    if os.path.getsize(path) == 0:
        img_file = file.replace(".txt", ".jpg")  # png면 png로 수정
        os.remove(path)
        os.remove(os.path.join(image_dir, img_file))

print("삭제 완료")

삭제 완료


In [21]:
import os

def check_labels(label_dir, nc=44):
    empty = []
    wrong_class = []
    wrong_format = []

    for file in os.listdir(label_dir):
        path = os.path.join(label_dir, file)

        if os.path.getsize(path) == 0:
            empty.append(file)
            continue

        with open(path) as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()

            # 최소 segmentation 형식 체크 (class + 최소 6개 좌표)
            if len(parts) < 7:
                wrong_format.append(file)
                continue

            cls = int(parts[0])
            if cls >= nc:
                wrong_class.append((file, cls))

    print("빈 라벨:", len(empty))
    print("형식 오류:", len(wrong_format))
    print("클래스 오류:", len(wrong_class))

    return empty, wrong_format, wrong_class


print("=== TRAIN ===")
check_labels("/content/drive/MyDrive/please/train/labels")

=== TRAIN ===


IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/please/train/labels/.ipynb_checkpoints'

In [16]:
import os

def find_wrong_labels(label_dir):
    wrong_format = []

    for file in os.listdir(label_dir):
        path = os.path.join(label_dir, file)

        with open(path) as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 7:  # segmentation 최소 조건
                wrong_format.append(file)
                break

    return wrong_format


wrong_files = find_wrong_labels("/content/drive/MyDrive/please/train/labels")
print("형식 오류 파일들:")
print(wrong_files)

형식 오류 파일들:
['Img_009_0327_jpg.rf.261f2242dc3a23735a67cd6535167db3.txt', 'Img_009_0305_jpg.rf.e3ffdd8b22c402fee512e5ebdb79c052.txt', 'Img_009_0327_jpg.rf.f607536366e065d85eb7d0d10e3fce28.txt', 'Img_009_0327_jpg.rf.1ae605c9b9a0724f88b89c00be3c5a91.txt', 'Img_009_0305_jpg.rf.561d5cf686c2fdd0744f781e4006abaa.txt', 'Img_009_0305_jpg.rf.54d0a4d4ea2ceb52ff1dbad7bdba9b27.txt', 'Img_079_0403_jpg.rf.bb5cbcd1b4848a8d65fa8c8615159676.txt', 'Img_079_0403_jpg.rf.bbea7b80a231ff6e82f7ce455254ad46.txt', 'Img_079_0403_jpg.rf.02076c85e9259a0534edcdb4c39a083f.txt']
